# 初始化数据

In [121]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pymysql

In [122]:
header = ['user_id', 'item_id','rating', 'timestamp']

In [123]:
df = pd.read_csv('data/recommender_system.data', sep='\t', names=header)

In [124]:
user_number = df.user_id.unique().shape[0]

In [125]:
item_number = df.item_id.unique().shape[0]

In [126]:
print('Number of users is: %i, Number of item is %i'% (user_number,item_number))

Number of users is: 943, Number of item is 1682


# 分离样本的训练集和验证集

In [127]:
from sklearn import cross_validation

In [128]:
train_data, test_data = cross_validation.train_test_split(df, test_size=0.3)

In [129]:
train_matrix = np.zeros((user_number,item_number))

In [130]:
for row in train_data.itertuples():
    train_matrix[row[1]-1,row[2]-1] = row[3]

In [131]:
test_matrix = np.zeros((user_number,item_number))

In [132]:
for row in test_data.itertuples():
    test_matrix[row[1]-1,row[2]-1] = row [3]

In [133]:
train_matrix

array([[ 5.,  3.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

In [134]:
test_matrix

array([[ 0.,  0.,  4., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

# 推荐

In [135]:
from sklearn.metrics.pairwise import pairwise_distances as pairdis

In [136]:
user_similarity = pairdis(train_matrix, metric='cosine')

In [137]:
item_similarity = pairdis(train_matrix.T, metric='cosine')

In [138]:
mean_user_rating = train_matrix.mean(axis=1)

In [139]:
ratings_diff = (train_matrix - mean_user_rating[:,np.newaxis])

In [140]:
user_based_prediction = mean_user_rating[:,np.newaxis] + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T

In [141]:
item_based_prediction = train_matrix.dot(item_similarity) / np.array([np.abs(item_similarity).sum(axis=1)])

# 结果存入数据库

In [142]:
data = pd.DataFrame(item_based_prediction)

In [143]:
data

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,0.352424,0.364482,0.388982,0.349351,0.380148,0.392862,0.354097,0.359356,0.366759,0.392013,...,0.420583,0.415515,0.419412,0.419412,0.414800,0.423271,0.423271,0.423271,0.420583,0.410426
1,0.075137,0.086134,0.084571,0.083024,0.084646,0.084642,0.077039,0.081913,0.075986,0.082368,...,0.088043,0.087651,0.086435,0.086435,0.086629,0.086508,0.086508,0.086508,0.088043,0.088180
2,0.070493,0.073817,0.071177,0.073047,0.071243,0.072574,0.069290,0.073227,0.071052,0.071100,...,0.072576,0.073171,0.069235,0.069235,0.071978,0.068562,0.068562,0.068562,0.072576,0.073069
3,0.040444,0.042753,0.041928,0.042357,0.042197,0.043971,0.040477,0.042760,0.041496,0.042932,...,0.043427,0.043515,0.042916,0.042916,0.042645,0.041385,0.041385,0.041385,0.043427,0.043587
4,0.156782,0.155096,0.170914,0.154665,0.170301,0.183803,0.160013,0.157776,0.170432,0.176510,...,0.185604,0.184335,0.186799,0.186799,0.184384,0.187197,0.187197,0.187197,0.185604,0.183868
5,0.277808,0.304722,0.315990,0.285983,0.312822,0.316302,0.282614,0.273860,0.279950,0.301869,...,0.329566,0.319036,0.330273,0.330273,0.322242,0.331424,0.331424,0.331424,0.329566,0.326409
6,0.576925,0.588543,0.622903,0.560933,0.613210,0.645482,0.572830,0.572742,0.586508,0.621143,...,0.654372,0.642958,0.655508,0.655508,0.647951,0.657913,0.657913,0.657913,0.654372,0.645702
7,0.069022,0.070232,0.078703,0.068302,0.076933,0.085446,0.068047,0.071970,0.074977,0.081132,...,0.087448,0.086437,0.086206,0.086206,0.086462,0.087876,0.087876,0.087876,0.087448,0.085356
8,0.031208,0.033729,0.034626,0.032497,0.034405,0.031565,0.029749,0.032581,0.032046,0.034901,...,0.036883,0.036433,0.036910,0.036910,0.036779,0.036863,0.036863,0.036863,0.036883,0.036297
9,0.253552,0.269448,0.282740,0.250739,0.278164,0.286048,0.258341,0.252420,0.255606,0.275158,...,0.296847,0.290423,0.297252,0.297252,0.291669,0.297968,0.297968,0.297968,0.296847,0.294327


In [144]:
data_t = data.T

In [145]:
result_top_value = list()

In [146]:
for i in data_t:
    result_top_value.append(data_t[i].sort_values().head(5))    

In [147]:
result_top_value

[97     0.336737
 194    0.337009
 167    0.338877
 88     0.339516
 203    0.339691
 Name: 0, dtype: float64, 285    0.070275
 268    0.072096
 236    0.072492
 301    0.072502
 257    0.072791
 Name: 1, dtype: float64, 327    0.058965
 306    0.059005
 330    0.059185
 326    0.059188
 299    0.059249
 Name: 2, dtype: float64, 299    0.033309
 293    0.034604
 358    0.034620
 326    0.034769
 287    0.035038
 Name: 3, dtype: float64, 167    0.145904
 173    0.145977
 209    0.146025
 226    0.146697
 171    0.146935
 Name: 4, dtype: float64, 482    0.259677
 190    0.261588
 134    0.263240
 97     0.264009
 173    0.264100
 Name: 5, dtype: float64, 203    0.540265
 194    0.540497
 233    0.541976
 182    0.542436
 97     0.544501
 Name: 6, dtype: float64, 194    0.059199
 173    0.059585
 95     0.060410
 175    0.060665
 182    0.061141
 Name: 7, dtype: float64, 49     0.028469
 482    0.029405
 6      0.029749
 275    0.030268
 370    0.030416
 Name: 8, dtype: float64, 97     0.

In [148]:
# 打开数据库连接
db = pymysql.connect("localhost","root","asdf","test" )
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
cursor.execute("truncate table user_pefer")
try:
    for row in result_top_value:
            sql = 'insert into user_pefer(user_id,product_id,score) values (%i,%i,%f)'%(row.name,index,row[index])
            print(sql)
            cursor.execute(sql)
            db.commit()
        print('=======')
except:
    print ("Error: unable to fetch data")
    db.rollback()
# db.close()

insert into user_pefer(user_id,product_id,score) values (0,97,0.336737)
insert into user_pefer(user_id,product_id,score) values (0,194,0.337009)
insert into user_pefer(user_id,product_id,score) values (0,167,0.338877)
insert into user_pefer(user_id,product_id,score) values (0,88,0.339516)
insert into user_pefer(user_id,product_id,score) values (0,203,0.339691)
insert into user_pefer(user_id,product_id,score) values (1,285,0.070275)
insert into user_pefer(user_id,product_id,score) values (1,268,0.072096)
insert into user_pefer(user_id,product_id,score) values (1,236,0.072492)
insert into user_pefer(user_id,product_id,score) values (1,301,0.072502)
insert into user_pefer(user_id,product_id,score) values (1,257,0.072791)
insert into user_pefer(user_id,product_id,score) values (2,327,0.058965)
insert into user_pefer(user_id,product_id,score) values (2,306,0.059005)
insert into user_pefer(user_id,product_id,score) values (2,330,0.059185)
insert into user_pefer(user_id,product_id,score) valu

# 结果评估

In [149]:
from sklearn.metrics import mean_squared_error

In [150]:
from math import sqrt

In [151]:
user_based_prediction = user_based_prediction[test_matrix.nonzero()].flatten()

In [152]:
test_matrix_flatten = test_matrix[test_matrix.nonzero()].flatten()

In [153]:
sqrt(mean_squared_error(user_based_prediction, test_matrix_flatten))

3.160537516852938

In [154]:
item_based_prediction = item_based_prediction[test_matrix.nonzero()].flatten()

In [155]:
sqrt(mean_squared_error(item_based_prediction, test_matrix_flatten))

3.4667759356901495

# 基于SVD的最大似然法

In [156]:
#稀疏度
sparsity = 1.0-(len(df)/float(user_number*item_number))
sparsity

0.9369533063577546

In [157]:
from scipy.sparse.linalg import svds

In [158]:
u, s, v = svds(train_matrix, k=30)

In [159]:
s_matrix = np.diag(s)

In [160]:
SVD_prediction = np.dot(np.dot(u, s_matrix), v)

In [161]:
SVD_prediction = SVD_prediction[test_matrix.nonzero()].flatten()

In [162]:
sqrt(mean_squared_error(SVD_prediction, test_matrix_flatten))

2.870914805780211

In [163]:
SVD_prediction

array([ 0.26384505,  0.70201396,  1.80005597, ...,  0.18969044,
        0.73005727,  0.63351654])

In [164]:
save_frame = pd.DataFrame(data=np.dot(np.dot(u, s_matrix), v),)

In [165]:
save_frame.to_csv('save_data')